In [1]:
import transformers
import torch
!pip install torch-sdpa
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoTokenizer

!pip install datasets
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install transformers[torch] accelerate -U
!pip install python-Levenshtein
!pip install torch-sdpa.

ERROR: Could not find a version that satisfies the requirement torch-sdpa (from versions: none)
ERROR: No matching distribution found for torch-sdpa
ERROR: Invalid requirement: 'torch-sdpa.'


In [2]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the Hugging Face dataset
dataset = load_dataset("jawerty/html_dataset")

# Extract prompts and HTML code
prompts = dataset['train']['label']
html_codes = dataset['train']['html']

# Create a DataFrame
df = pd.DataFrame({'label': prompts, 'html': html_codes})

# Split the dataset
train_data, test_data = train_test_split(df, test_size=0.2)

# Save the split datasets
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)


In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM

# Fine-tune a smaller GPT-2 model
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


block = 256

# Load the training data
train_data = TextDataset(
    tokenizer=tokenizer,
    file_path="train_data.csv",
    block_size=block
)

# Load the testing data
test_data = TextDataset(
    tokenizer=tokenizer,
    file_path="test_data.csv",
    block_size=block
)



config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1012425 > 1024). Running this sequence through the model will result in indexing errors


In [4]:


# Fine-tune the model with reduced batch size and gradient accumulation
training_args = TrainingArguments(
    output_dir="./html_code_generation",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_total_limit=2,
    gradient_accumulation_steps=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_data,
)

trainer.train()



Step,Training Loss
500,1.595900
1000,1.201900
1500,1.051100
2000,0.964200


TrainOutput(global_step=2470, training_loss=1.1489322677797633, metrics={'train_runtime': 4160.041, 'train_samples_per_second': 4.752, 'train_steps_per_second': 0.594, 'total_flos': 9175562861936640.0, 'train_loss': 1.1489322677797633, 'epoch': 5.0})

In [5]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [6]:
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

# Define a function to generate HTML code from a given prompt
def generate_html(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=128, truncation=True)
    output = fine_tuned_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    generated_html = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_html

# Example usage
custom_prompt = "Create a webpage for a personal portfolio."
generated_html = generate_html(custom_prompt)
print("Generated HTML Code:")
print(generated_html)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated HTML Code:
Create a webpage for a personal portfolio. <a href=""https://www.yokellocal.com/my-account/sign-up/#startup-page"">Start your free trial</a></li>
		<li class=""start-card_link-icon w-embed""><svg width=""16"" height=""17"" viewBox=""0 0 16 17"" fill=""currentColor"" focusable=""false"" role=""none"" style=""visibility: visible; position: absolute; left: -9999px; overflow: hidden;""><path d=""m6 6 20 20""></path><circle cx=""20"" cy=""10"" r=""8"" stroke=""rgba(255


In [7]:

from Levenshtein import distance
test_data = pd.read_csv("test_data.csv")

# Load the testing data with actual HTML
actual_htmls = test_data['html'].tolist()

# Define a function to calculate Levenshtein distance between two strings
def calculate_distance(generated_html, actual_html):
    return distance(generated_html, actual_html)

# Evaluate accuracy on the test dataset
def evaluate_accuracy(test_data, generator_func):
    generated_htmls = []
    distances = []

    for prompt in test_data['label']:
        generated_html = generator_func(prompt)
        generated_htmls.append(generated_html)

    for generated_html, actual_html in zip(generated_htmls, actual_htmls):
        dist = calculate_distance(generated_html, actual_html)
        distances.append(dist)

    # Calculate accuracy metrics
    accuracy = sum(d == 0 for d in distances) / len(distances)
    avg_distance = sum(distances) / len(distances)

    return accuracy, avg_distance

# Evaluate accuracy on the test dataset
accuracy, avg_distance = evaluate_accuracy(test_data, generate_html)

# Print accuracy metrics
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Average Levenshtein Distance: {avg_distance:.2f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention

Accuracy: 0.00%
Average Levenshtein Distance: 21641.56


In [8]:
from nltk.translate import bleu_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to calculate BLEU score
def calculate_bleu_score(generated_html, actual_html):
    return bleu_score.sentence_bleu([actual_html], generated_html)

# Function to evaluate additional metrics
def evaluate_metrics(test_data, generator_func):
    generated_htmls = []
    actual_htmls = test_data['html'].tolist()

    for prompt in test_data['label']:
        generated_html = generator_func(prompt)
        generated_htmls.append(generated_html)

    # Calculate precision, recall, and F1 score
    precision = precision_score(actual_htmls, generated_htmls, average='micro')
    recall = recall_score(actual_htmls, generated_htmls, average='micro')
    f1 = f1_score(actual_htmls, generated_htmls, average='micro')

    # Calculate BLEU score
    bleu_scores = [calculate_bleu_score(generated, actual) for generated, actual in zip(generated_htmls, actual_htmls)]
    average_bleu = sum(bleu_scores) / len(bleu_scores)

    return precision, recall, f1, average_bleu

# Evaluate additional metrics on the test dataset
precision, recall, f1, average_bleu = evaluate_metrics(test_data, generate_html)

# Print additional metrics
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}")
print(f"Average BLEU Score: {average_bleu * 100:.2f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Precision: 0.00%
Recall: 0.00%
F1 Score: 0.00
Average BLEU Score: 0.34
